In [1]:
# 104_job_bank with Selenium (Overview)
# import packages
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import csv , time

# Get Driver
service = Service("./chromedriver")
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
driver = Chrome(service=service,options = chrome_options)

# define get

In [2]:
# 104 home page's url
url = "https://www.104.com.tw/jobs/main/"  # go to jobbank
driver.get(url) # surf jobbank with browser
driver.maximize_window() # maximize broswer window -> get more HTML ASAP
time.sleep(3)

In [3]:
# Type KeyWord of interest
driver.find_element(by=By.XPATH, value='/html/body/article[1]/div/div/div[4]/div/input').send_keys("光學工程師")
time.sleep(3)

In [4]:
# Push search button
driver.find_element(by=By.XPATH,value='/html/body/article[1]/div/div/div[4]/div/button').click()
time.sleep(3)

In [5]:
# # Scroll page 4 times
# for i in range(4):
#     time.sleep(2)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [6]:
# find HTML element including the jobs' info
# articles = driver.find_elements(By.CLASS_NAME, "b-block--top-bord.job-list-item.b-clearfix.js-job-item")
jobs = driver.find_elements(By.CLASS_NAME, "js-job-link")
jobs_numbers = len(jobs)

In [7]:
# find job details in articles which has same form individually
for nth_job in range(jobs_numbers) :
    # # get value coressponding to attribute (key)
    # articles = driver.find_elements(By.CLASS_NAME, "b-block--top-bord.job-list-item.b-clearfix.js-job-item")
    # find nth-article's job-link
    job_name = driver.find_elements(By.CLASS_NAME, "js-job-link")[nth_job]
    # get value coressponding to attribute (key)
    job_link = job_name.get_attribute("href")
    print(job_name.text)
    print(job_link)
    driver.get(job_link)
    time.sleep(3)
    driver.back()
    time.sleep(5)

半導體設備工程師(桃園)
https://www.104.com.tw/job/6ig8h?jobsource=hotjob_chr
精密機構/機械
https://www.104.com.tw/job/4hvfp?jobsource=hotjob_chr
14P-T-0002光學工程師
https://www.104.com.tw/job/5ep52?jobsource=jolist_c_relevance
光學工程師
https://www.104.com.tw/job/4mzl6?jobsource=jolist_c_relevance
視光學工程師
https://www.104.com.tw/job/5b3c0?jobsource=jolist_c_relevance
車燈光學/機構設計工程師
https://www.104.com.tw/job/700vm?jobsource=jolist_c_relevance
光學工程師
https://www.104.com.tw/job/50puf?jobsource=jolist_c_relevance
研發類-LiDAR Light Engine Sr. Mechanical Engineer 光達光機機構設計資深工程師(CMC)
https://www.104.com.tw/job/5mb1d?jobsource=jolist_c_relevance
資深工程師 (背光模組研發)
https://www.104.com.tw/job/7watg?jobsource=jolist_c_relevance
工研院材化所_光電工程師(M100)
https://www.104.com.tw/job/7tl5a?jobsource=jolist_c_relevance
光學資深工程師Senior Engineer (Optical field)
https://www.104.com.tw/job/7lkfq?jobsource=jolist_c_relevance
資深非成像光學設計人員_11801
https://www.104.com.tw/job/7sc9q?jobsource=jolist_c_relevance
Micro LED面板開發工程師_2023預聘暨研發替代役(新鮮人限定/新竹)R07
htt